In [3]:
import ee
import pandas as pd
import geemap

# -----------------------------------------------------------------------------
# 1. INITIALIZATION & CONFIGURATION
# -----------------------------------------------------------------------------
geemap.ee_initialize()

START_PERIOD = [1985, 1987]
END_PERIOD = [2022, 2024]
SEASON_MONTHS = [6, 7, 8]  # Summer months

# Florida counties FeatureCollection
FLORIDA_COUNTIES = ee.FeatureCollection("TIGER/2018/Counties").filter(
    ee.Filter.eq('STATEFP', '12')
)

THERMAL_BANDS_L8_9 = 'ST_B10'
THERMAL_BANDS_L5_7 = 'ST_B6'

# -----------------------------------------------------------------------------
# 2. CLOUD MASK & LST CONVERSION
# -----------------------------------------------------------------------------
def mask_landsat_clouds(image):
    qa = image.select('QA_PIXEL')
    cloud_mask = (1 << 3) | (1 << 5)
    mask = qa.bitwiseAnd(cloud_mask).eq(0)
    return image.updateMask(mask)

def get_lst_celsius(image):
    is_l8_or_l9 = ee.List(['LANDSAT_8', 'LANDSAT_9']).contains(image.get('SPACECRAFT_ID'))
    st_band_name = ee.String(
        ee.Algorithms.If(is_l8_or_l9, THERMAL_BANDS_L8_9, THERMAL_BANDS_L5_7)
    )
    lst_celsius = image.select(st_band_name) \
                       .multiply(0.00341802) \
                       .add(149.0) \
                       .subtract(273.15)
    return image.addBands(lst_celsius.rename('LST'))

# -----------------------------------------------------------------------------
# 3. PERIOD LST CALCULATION
# -----------------------------------------------------------------------------
def analyze_lst_for_period(period):
    start_date = ee.Date.fromYMD(period[0], 1, 1)
    end_date = ee.Date.fromYMD(period[1], 12, 31)

    l9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
    l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    l7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
    l5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')

    landsat_collection = l9.merge(l8).merge(l7).merge(l5)

    image_composite = (landsat_collection
                       .filterBounds(FLORIDA_COUNTIES.geometry())
                       .filterDate(start_date, end_date)
                       .filter(ee.Filter.calendarRange(SEASON_MONTHS[0], SEASON_MONTHS[-1], 'month'))
                       .map(mask_landsat_clouds)
                       .map(get_lst_celsius)
                       .select('LST')
                       .mean())
    return image_composite

# -----------------------------------------------------------------------------
# 4. DATA AGGREGATION
# -----------------------------------------------------------------------------
print("🛰 Calculating summer LST composites...")
start_lst_composite = analyze_lst_for_period(START_PERIOD)
end_lst_composite = analyze_lst_for_period(END_PERIOD)

print("⚙ Aggregating county statistics...")
start_stats = start_lst_composite.reduceRegions(
    collection=FLORIDA_COUNTIES, reducer=ee.Reducer.mean(), scale=100
)
end_stats = end_lst_composite.reduceRegions(
    collection=FLORIDA_COUNTIES, reducer=ee.Reducer.mean(), scale=100
)

df_start = geemap.ee_to_df(start_stats)[['NAME', 'mean']]
df_end = geemap.ee_to_df(end_stats)[['NAME', 'mean']]

df_start.columns = ['County', f'LST {START_PERIOD[0]}-{START_PERIOD[1]} (°C)']
df_end.columns = ['County', f'LST {END_PERIOD[0]}-{END_PERIOD[1]} (°C)']

final_df = pd.merge(df_start, df_end, on='County')
final_df['LST Growth (°C)'] = final_df[f'LST {END_PERIOD[0]}-{END_PERIOD[1]} (°C)'] - \
                              final_df[f'LST {START_PERIOD[0]}-{START_PERIOD[1]} (°C)']
final_df = final_df.dropna()

# -----------------------------------------------------------------------------
# 5. TOP/BOTTOM LISTS
# -----------------------------------------------------------------------------
highest_lst = final_df.nlargest(5, f'LST {END_PERIOD[0]}-{END_PERIOD[1]} (°C)')
lowest_lst = final_df.nsmallest(5, f'LST {END_PERIOD[0]}-{END_PERIOD[1]} (°C)')
highest_growth = final_df.nlargest(5, 'LST Growth (°C)')
lowest_growth = final_df.nsmallest(5, 'LST Growth (°C)')

# -----------------------------------------------------------------------------
# 6. INTERACTIVE MAP VISUALIZATION
# -----------------------------------------------------------------------------
print("🗺 Displaying interactive map...")

m = geemap.Map(center=[28.5, -82.5], zoom=7)

lst_vis_params = {
    'min': final_df[f'LST {END_PERIOD[0]}-{END_PERIOD[1]} (°C)'].min(),
    'max': final_df[f'LST {END_PERIOD[0]}-{END_PERIOD[1]} (°C)'].max(),
    'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
}

m.addLayer(end_lst_composite.clip(FLORIDA_COUNTIES.geometry()),
           lst_vis_params,
           f'Mean Summer LST {END_PERIOD[0]}-{END_PERIOD[1]} (°C)')

# Highlight layers
m.addLayer(FLORIDA_COUNTIES.filter(ee.Filter.inList('NAME', highest_lst['County'].tolist())),
           {'color': '#FF0000', 'fillColor': '#FF000088'},
           '🟥 Top 5 Highest LST')

m.addLayer(FLORIDA_COUNTIES.filter(ee.Filter.inList('NAME', lowest_lst['County'].tolist())),
           {'color': '#0000FF', 'fillColor': '#0000FF88'},
           '🟦 Top 5 Lowest LST')

m.addLayer(FLORIDA_COUNTIES.filter(ee.Filter.inList('NAME', highest_growth['County'].tolist())),
           {'color': '#FFA500', 'fillColor': '#FFA50088'},
           '📈 Top 5 Highest Growth')

m.addLayer(FLORIDA_COUNTIES.filter(ee.Filter.inList('NAME', lowest_growth['County'].tolist())),
           {'color': '#00FFFF', 'fillColor': '#00FFFF88'},
           '📉 Top 5 Lowest Growth')

# Add controls
m.add_colorbar(lst_vis_params, label="Mean Summer LST (°C)")
m.add_layer_control()

m


🛰 Calculating summer LST composites...
⚙ Aggregating county statistics...
🗺 Displaying interactive map...


Map(center=[28.5, -82.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…